In [54]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm as tqdmflat

import spacy
from spacy.lang.en import English
import os

#from simpletransformers.classification import ClassificationModel

import logging
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split, StratifiedKFold

import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")




Version:  2.1.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [3]:
PATH = 'data/'
biz=pd.read_csv(os.path.join(PATH,"yelp_business.csv"))
reviews=pd.read_csv(os.path.join(PATH,"yelp_review.csv"))

In [4]:

col = ['neighborhood', 'address', 'latitude', 'longitude', 'stars']
biz.drop(columns=col, inplace=True)    

biz.categories = biz.categories.apply(lambda x: x.split(";"))

biz = biz[
    (biz.city == 'Cleveland') & 
    (biz.state == 'OH') & 
    (biz.is_open == 1) & 
    (biz.categories.apply(
        lambda x: True if 'Restaurants' in x else False
    ))]


In [5]:
df = pd.merge(reviews, biz, how='inner', on='business_id')

In [6]:
df.head(2)

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,name,city,state,postal_code,review_count,is_open,categories
0,OPZsR2jCG72uoDNjU71DQQ,qYbWTWH5leltA0bzWAOnmA,meXjqyhTNLFmknY39y2sMg,5,2014-09-11,Solid beers -- Christmas Ale defines my holida...,1,1,1,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv..."
1,fxGwEiSYDtAen8BNuVGGxg,8Az_JgEpXqAii_5EDkw2tw,meXjqyhTNLFmknY39y2sMg,3,2013-10-13,Meh. It was OK. A bartender the night before...,0,1,0,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv..."


In [15]:
df = df.assign(
    target= (df.stars >= 4).astype(int)
)
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,name,city,state,postal_code,review_count,is_open,categories,target
0,OPZsR2jCG72uoDNjU71DQQ,qYbWTWH5leltA0bzWAOnmA,meXjqyhTNLFmknY39y2sMg,5,2014-09-11,Solid beers -- Christmas Ale defines my holida...,1,1,1,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv...",1
1,fxGwEiSYDtAen8BNuVGGxg,8Az_JgEpXqAii_5EDkw2tw,meXjqyhTNLFmknY39y2sMg,3,2013-10-13,Meh. It was OK. A bartender the night before...,0,1,0,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv...",0
2,Gweb4pADeQ26WnaiKEZ7GQ,T9tEic49JZjN4nCUcDvrRQ,meXjqyhTNLFmknY39y2sMg,4,2014-01-15,"Oh Christmas Ale, oh Christmas Ale, how lovely...",1,1,1,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv...",1
3,P1vhwPI56SeZEz10ywaS7w,W1p8_CFW5FISSihmQo5Qzw,meXjqyhTNLFmknY39y2sMg,3,2012-02-09,What is the big deal about this place? The foo...,2,1,1,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv...",0
4,1kQvQlBX0V5_rGddBh9-rQ,Y_PP05RRdzbKRYfDCCfh8w,meXjqyhTNLFmknY39y2sMg,5,2017-04-30,Great Lakes Brewing Company is one of my favor...,0,0,0,"""Great Lakes Brewing Company""",Cleveland,OH,44113,751,1,"[Breweries, Restaurants, Event Planning & Serv...",1


In [18]:
# Declare X & Y
Y = df['target']
X = df['text']

# Let's stratify the data so we get a fair balance
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42, stratify=Y, shuffle=True
)

X_trn, X_val, y_trn, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=2718, stratify=y_train, shuffle=True
)

In [29]:
trn_dataset = tf.data.Dataset.from_tensor_slices((X_trn.values, y_trn.values))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
full_trn_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))



In [30]:
next(iter(trn_dataset))

(<tf.Tensor: shape=(), dtype=string, numpy=b"Had breakfast here this morning with my husband, my 1.5 year old daughter, my sister-in-law, and brother-in-law who were visiting from Atlanta.  The food was great, the servers were very friendly and attentive, and we loved the art on the walls.  I particularly loved the random collection of coffee mugs they use.  Makes you feel like you are sipping coffee at your kitchen table at home!  There is also outdoor patio seating and they have a very good sized parking lot so you don't have to struggle to find street parking.  A great experience, and we will definitely be back!">,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [55]:
#embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
embeddingg = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2'
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(next(iter(trn_dataset.batch(10)))[0][:3])


<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 3.1346078e+00, -1.4341266e+00,  4.3682310e-01,  6.8576205e-01,
        -3.0143085e+00, -2.3653281e+00, -1.7773513e+00,  1.7121491e+00,
         1.2667683e+00,  7.2778749e-01, -4.2402697e+00,  2.9655933e+00,
        -2.2652957e+00, -1.2820994e-02, -6.4009433e+00,  1.8368458e-02,
         3.6295979e+00, -3.4151301e-01, -2.3160038e+00, -8.6310530e-01],
       [ 2.8281808e-01,  6.1656821e-01,  2.3511133e-01, -1.2997293e+00,
        -1.0978019e+00, -5.0215811e-01, -5.8447570e-01, -7.9597253e-01,
         4.9229687e-01,  9.0782791e-03,  1.3185677e-01,  3.3502468e-01,
        -1.0797229e+00,  4.0779716e-01, -2.4878178e+00,  1.8422601e-01,
         1.0713542e+00, -9.7727813e-02, -1.0575348e+00, -2.6321512e-01],
       [ 4.1388006e+00, -3.6240318e+00,  3.4785690e+00,  4.2542901e+00,
        -1.0087058e+01, -7.5313406e+00, -4.8082728e+00,  2.9591877e+00,
         6.0119114e+00, -8.6026907e-01, -4.2112665e+00,  6.2156134e+00,
        -2.92

In [53]:
next(iter(trn_dataset.batch(10)))[0]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"Had breakfast here this morning with my husband, my 1.5 year old daughter, my sister-in-law, and brother-in-law who were visiting from Atlanta.  The food was great, the servers were very friendly and attentive, and we loved the art on the walls.  I particularly loved the random collection of coffee mugs they use.  Makes you feel like you are sipping coffee at your kitchen table at home!  There is also outdoor patio seating and they have a very good sized parking lot so you don't have to struggle to find street parking.  A great experience, and we will definitely be back!",
       b"I was born in Korea and raised in LA and NYC. \n\nHa Ahn in Cleveland is one of the Top Five Korean restaurants I've ever been to.\n\nTruly authentic and every ingredient seems to be of the highest quality, and incredibly fresh.",
       b'This review is so difficult for me to write since I went there on my anniversary and I don\'t want to make it sound 

In [56]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [58]:
history = model.fit(trn_dataset.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=val_dataset.batch(512),
                    verbose=1)


Train for 76 steps, validate for 9 steps
Epoch 1/20
76/76 [==============================] - 3s 37ms/step - loss: 0.6615 - accuracy: 0.5527 - val_loss: 0.5714 - val_accuracy: 0.6634
Epoch 2/20
76/76 [==============================] - 2s 27ms/step - loss: 0.5141 - accuracy: 0.7231 - val_loss: 0.4705 - val_accuracy: 0.7559
Epoch 3/20
76/76 [==============================] - 2s 28ms/step - loss: 0.4254 - accuracy: 0.8017 - val_loss: 0.4037 - val_accuracy: 0.8128
Epoch 4/20
76/76 [==============================] - 2s 29ms/step - loss: 0.3688 - accuracy: 0.8371 - val_loss: 0.3637 - val_accuracy: 0.8374
Epoch 5/20
76/76 [==============================] - 2s 29ms/step - loss: 0.3304 - accuracy: 0.8574 - val_loss: 0.3375 - val_accuracy: 0.8538
Epoch 6/20
76/76 [==============================] - 2s 28ms/step - loss: 0.3061 - accuracy: 0.8713 - val_loss: 0.3167 - val_accuracy: 0.8557
Epoch 7/20
76/76 [==============================] - 2s 28ms/step - loss: 0.2897 - accuracy: 0.8801 - val_loss: 0.

In [59]:
results = model.evaluate(test_dataset.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))


28/28 - 1s - loss: 0.2928 - accuracy: 0.8834
loss: 0.293
accuracy: 0.883


In [61]:
model2 = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


ValueError: Input 0 of layer bidirectional_2 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 20]